In [12]:
from dask import dataframe as dd

In [13]:
cols = ['Covered_Recipient_Type','Teaching_Hospital_CCN','Teaching_Hospital_ID','Teaching_Hospital_Name',
       'Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name','Physician_Last_Name','Physician_Name_Suffix',
       'Recipient_State','Physician_Primary_Type','Physician_Specialty','Physician_License_State_code1','Physician_License_State_code2',
       'Physician_License_State_code3','Physician_License_State_code4','Physician_License_State_code5',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Total_Amount_of_Payment_USDollars','Date_of_Payment','Number_of_Payments_Included_in_Total_Amount',
       'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value','City_of_Travel',
       'State_of_Travel', 'Country_of_Travel','Physician_Ownership_Indicator','Third_Party_Payment_Recipient_Indicator',
       'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value','Charity_Indicator','Third_Party_Equals_Covered_Recipient_Indicator',
       'Record_ID','Related_Product_Indicator','Covered_or_Noncovered_Indicator_1',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Product_Category_or_Therapeutic_Area_1','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Associated_Drug_or_Biological_NDC_1','Covered_or_Noncovered_Indicator_2',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Product_Category_or_Therapeutic_Area_2',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Associated_Drug_or_Biological_NDC_2',
       'Covered_or_Noncovered_Indicator_3',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Product_Category_or_Therapeutic_Area_3',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Associated_Drug_or_Biological_NDC_3',
       'Covered_or_Noncovered_Indicator_4',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Product_Category_or_Therapeutic_Area_4',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Associated_Drug_or_Biological_NDC_4',
       'Covered_or_Noncovered_Indicator_5',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Product_Category_or_Therapeutic_Area_5',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Associated_Drug_or_Biological_NDC_5','Recipient_Zip_Code']

In [14]:
#Load the data
sunshine = dd.read_csv('../Data/OP_DTL_GNRL_PGYR2019_P06302021.csv',dtype=str,usecols=cols)

In [15]:
# Dealing with missing data for recipient state
def licensedCA(row):
    if row['Recipient_State']=='CA' or row['Physician_License_State_code1']=='CA' or row['Physician_License_State_code1']=='CA' or row['Physician_License_State_code2']=='CA' or row['Physician_License_State_code3']=='CA' or row['Physician_License_State_code4']=='CA' or row['Physician_License_State_code5']=='CA':
        return True
    return False

sunshine['Licensed_In_CA'] = sunshine.apply(licensedCA,axis=1,meta=str)
        

In [16]:
%%time
'''
1. Filter only for California 
2. Persisting `cadf` in memory so we don't need to repeat this filtering step every time we 
   want to create a new output file
'''
cadf = sunshine[sunshine['Licensed_In_CA']==True]
cadf = cadf.persist()
del sunshine

CPU times: user 3min 49s, sys: 6.56 s, total: 3min 55s
Wall time: 3min 26s


In [17]:
%%time
# Hospital ID and Payment ID
hosp = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Teaching Hospital'][['Record_ID','Teaching_Hospital_ID']]
hosp.to_csv('../Data/Outputs_Cleanup/Sunshine/hospitals_payment.csv',single_file=True,index=False)

CPU times: user 176 ms, sys: 44.1 ms, total: 220 ms
Wall time: 252 ms


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/hospitals_payment.csv']

In [18]:
%%time
# Hospitals and states
hosp = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Teaching Hospital'][['Recipient_State','Teaching_Hospital_ID']]
hosp = hosp.drop_duplicates(subset=['Teaching_Hospital_ID'])
hosp.to_csv('../Data/Outputs_Cleanup/Sunshine/hosp_states.csv',single_file=True,index=False)

CPU times: user 199 ms, sys: 13.5 ms, total: 212 ms
Wall time: 206 ms


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/hosp_states.csv']

In [19]:
%%time
# Hospitals' other information
hosp = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Teaching Hospital'][['Teaching_Hospital_CCN','Teaching_Hospital_ID','Teaching_Hospital_Name']]
hosp = hosp.drop_duplicates(subset=['Teaching_Hospital_ID'])
hosp.to_csv('../Data/Outputs_Cleanup/Sunshine/hosp_info.csv',single_file=True,index=False)
del hosp

CPU times: user 188 ms, sys: 12.7 ms, total: 201 ms
Wall time: 197 ms


In [20]:
%%time
# Physician ID and Payment ID
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Record_ID','Physician_Profile_ID']]
physicians.to_csv('../Data/Outputs_Cleanup/Sunshine/physicians_payments.csv',single_file=True,index=False)

CPU times: user 2.58 s, sys: 139 ms, total: 2.72 s
Wall time: 2.72 s


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/physicians_payments.csv']

In [21]:
%%time
# Physicians and states
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Recipient_State','Physician_Profile_ID']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])
physicians.to_csv('../Data/Outputs_Cleanup/Sunshine/physicians_state.csv',single_file=True,index=False)

CPU times: user 1.55 s, sys: 49.7 ms, total: 1.6 s
Wall time: 1.46 s


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/physicians_state.csv']

In [22]:
%%time
# Physicians other information
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name','Physician_Last_Name','Physician_Name_Suffix',
       'Recipient_State','Physician_Primary_Type','Physician_Specialty','Physician_License_State_code1','Physician_License_State_code2',
       'Physician_License_State_code3','Physician_License_State_code4','Physician_License_State_code5','Recipient_Zip_Code']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])
physicians.to_csv('../Data/Outputs_Cleanup/Sunshine/physicians_info.csv',single_file=True,index=False)
del physicians

CPU times: user 2.16 s, sys: 62.9 ms, total: 2.22 s
Wall time: 2.08 s


In [23]:
%%time
# Payment information
payments = cadf[['Record_ID','Total_Amount_of_Payment_USDollars','Date_of_Payment','Number_of_Payments_Included_in_Total_Amount',
                 'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value']]
payments.to_csv('../Data/Outputs_Cleanup/Sunshine/payments.csv',single_file=True,index=False)
del payments

CPU times: user 3.12 s, sys: 90.4 ms, total: 3.21 s
Wall time: 3.21 s


In [24]:
%%time
# Pharmco and Record ID
pharmCo = cadf[['Record_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID']]
pharmCo.to_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv',single_file=True,index=False)

CPU times: user 1.7 s, sys: 47.8 ms, total: 1.75 s
Wall time: 1.77 s


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv']

In [25]:
%%time
# Pharmco and state
pharmCo = cadf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State']]
pharmCo.to_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_states.csv',single_file=True,index=False)

CPU times: user 1.5 s, sys: 33.6 ms, total: 1.53 s
Wall time: 1.52 s


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/pharmCo_states.csv']

In [26]:
%%time
# Pharmco other information
pharmCo = cadf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID','Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
                'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name','Related_Product_Indicator']]
pharmCo.to_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_info.csv',single_file=True,index=False)

CPU times: user 2.66 s, sys: 88.1 ms, total: 2.75 s
Wall time: 2.78 s


['/Users/jonahbreslow/Documents/DSE/2020-jfbreslow/DSE203/DSE-203-Knowledge-Graph/Data Cleanup/../Data/Outputs_Cleanup/Sunshine/pharmCo_info.csv']

In [ ]:
%%time
# Pharmco other information
pharmCo = cadf[['Record_ID','Covered_or_Noncovered_Indicator_1',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1','Product_Category_or_Therapeutic_Area_1',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1','Associated_Drug_or_Biological_NDC_1','Covered_or_Noncovered_Indicator_2',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2','Product_Category_or_Therapeutic_Area_2',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2','Associated_Drug_or_Biological_NDC_2',
                'Covered_or_Noncovered_Indicator_3','Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
                'Product_Category_or_Therapeutic_Area_3','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
                'Associated_Drug_or_Biological_NDC_3','Covered_or_Noncovered_Indicator_4',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4','Product_Category_or_Therapeutic_Area_4',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4','Associated_Drug_or_Biological_NDC_4',
                'Covered_or_Noncovered_Indicator_5','Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                'Product_Category_or_Therapeutic_Area_5','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                'Associated_Drug_or_Biological_NDC_5']]
pharmCo.to_csv('../Data/Outputs_Cleanup/Sunshine/record_drugs.csv',single_file=True,index=False)